In [1]:
import numpy as np
import os
import fileinput
import sys
import pandas as pd
import matplotlib.pyplot as plt

from utilities.foam_automation.caserunner import run_until_convergence, get_endtime
from utilities.foam_automation.computeFoamLossFunc import computeFoamFieldLossFunc
from utilities.foam_automation.set_foam_coeff import set_foam_coef

from utilities.json_io import write_suggestion, write_json, load_coeff_default, load_coeff_bounds, load_history_loss_log, load_suggestion

from utilities.control import initialize
from update_history import register_score
from suggest_coeffs import suggest
import random

# Bayesian optimization

In [2]:
# Run simulation loop
foamdir = '/home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_phll/case_1p0_20iter'
savedir = '/home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_phll/case_1p0_20iter_opt'

tunerdir = os.path.join(foamdir,'tuner')
iterations=30
n_points_sampled = 4
utility = 'poi'
xi = 0.1
ref_df = pd.read_csv(os.path.join(tunerdir,'refdata.csv'))


initialize(directory = tunerdir,
           coeff_bounds = None,
           coeff_default = None,
           restart = True)

foamLoss = computeFoamFieldLossFunc(foamdir=foamdir,
                                    ref_df=ref_df,
                                    interp_method='nearest')
np.random.seed(7)
a1_rand = np.random.uniform(.24, .6, size=(n_points_sampled))
betaStar_rand = np.random.uniform(.045, .14, size=(n_points_sampled))
score_best = 1E6 

for i in range(iterations):
    if i == 0:
        search_point=load_coeff_default(directory=tunerdir)
        write_suggestion(directory=tunerdir, suggestion=search_point)
    elif i < n_points_sampled:
        search_point = {"a1": a1_rand[i], "betaStar": betaStar_rand[i]}
        write_suggestion(directory=tunerdir, suggestion=search_point)
    else:
        suggest(directory=tunerdir, random_state=7,utility_kind = utility, xi=xi)
    
    search_point = load_suggestion(directory=tunerdir)
    print(search_point)
    set_foam_coef(foamdir,search_point)
    last_time = run_until_convergence(case=foamdir, n_proc = 8)
    score = -foamLoss.foam_general_loss_function(foamtime=last_time,
                                                   coef_default_dict = load_coeff_default(directory=tunerdir),
                                                   coef_dict=search_point,
                                                   error_calc_fields=['U','k'],
                                                   error_calc_intparams=[],
                                                   error_type = 'mse'
                                                  )
                                                   
    register_score(score=score, directory=tunerdir)
    if abs(score) < abs(score_best):   
        score_best = score
        os.system(f'rm -r {savedir}')
        os.system(f'cp -r {foamdir} {savedir}')

    

Did not get coefficient bounds dict for initialization, assuming coeff_bounds.json exists in /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_phll/case_1p0_20iter/tuner
Did not get coefficient default dict for initialization, assuming coeff_default.json exists in /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_phll/case_1p0_20iter/tuner or is not needed
Removing old history and suggestion files....
Saving suggestion to /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_phll/case_1p0_20iter/tuner/suggestion.json
{'a1': 0.31, 'betaStar': 0.09}
error_term 6.113326585006639e-06
coef_term 0.0
score 6.113326585006639e-06
No history file exists at /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_phll/case_1p0_20iter/tuner/history.json, creating and registering first point
Registering score in /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_phll/case_1p0_20iter/tuner/history.json
Saving suggestion to /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_phl

rm: cannot remove '/home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_phll/case_1p0_20iter/processor*': No such file or directory


 Interrupted by the Keyboard
Killing PID 18856
error_term 6.11332979447776e-06
coef_term 0.5191463420121418
score 7.700186194636245e-06
Registering score in /home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_phll/case_1p0_20iter/tuner/history.json


rm: cannot remove '/home/ryley/ML/foam/run/turbo-rans/bayesian_a1_betaStar_phll/case_1p0_20iter/processor*': No such file or directory

KeyboardInterrupt

